First, let's load the JSON file which describes the human pose task.  This is in COCO format, it is the category descriptor pulled from the annotations file.  We modify the COCO category slightly, to add a neck keypoint.  We will use this task description JSON to create a topology tensor, which is an intermediate data structure that describes the part linkages, as well as which channels in the part affinity field each linkage corresponds to.

In [1]:
import json
import trt_pose.coco

with open('human_pose.json', 'r') as f:
    human_pose = json.load(f)

topology = trt_pose.coco.coco_category_to_topology(human_pose)

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-cd62gk4c because the default path (/home/aicoach/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Matplotlib is building the font cache; this may take a moment.


Next, we'll load our model.  Each model takes at least two parameters, *cmap_channels* and *paf_channels* corresponding to the number of heatmap channels
and part affinity field channels.  The number of part affinity field channels is 2x the number of links, because each link has a channel corresponding to the
x and y direction of the vector field for each link.

In [2]:
import trt_pose.models

num_parts = len(human_pose['keypoints'])
num_links = len(human_pose['skeleton'])

model = trt_pose.models.resnet18_baseline_att(num_parts, 2 * num_links).cuda().eval()

Next, let's load the model weights.  You will need to download these according to the table in the README.

In [3]:
import torch

MODEL_WEIGHTS = 'resnet18_baseline_att_224x224_A_epoch_249.pth'

model.load_state_dict(torch.load(MODEL_WEIGHTS))

<All keys matched successfully>

In order to optimize with TensorRT using the python library *torch2trt* we'll also need to create some example data.  The dimensions
of this data should match the dimensions that the network was trained with.  Since we're using the resnet18 variant that was trained on
an input resolution of 224x224, we set the width and height to these dimensions.

In [4]:
WIDTH = 224
HEIGHT = 224

data = torch.zeros((1, 3, HEIGHT, WIDTH)).cuda()

Next, we'll use [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) to optimize the model.  We'll enable fp16_mode to allow optimizations to use reduced half precision.

In [5]:
import torch2trt

model_trt = torch2trt.torch2trt(model, [data], fp16_mode=True, max_workspace_size=1<<25)

The optimized model may be saved so that we do not need to perform optimization again, we can just load the model.  Please note that TensorRT has device specific optimizations, so you can only use an optimized model on similar platforms.

In [6]:
OPTIMIZED_MODEL = 'resnet18_baseline_att_224x224_A_epoch_249_trt.pth'

torch.save(model_trt.state_dict(), OPTIMIZED_MODEL)

We could then load the saved model using *torch2trt* as follows.

In [7]:
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load(OPTIMIZED_MODEL))

<All keys matched successfully>

We can benchmark the model in FPS with the following code

In [8]:
#import time

#t0 = time.time()
#torch.cuda.current_stream().synchronize()
#for i in range(50):
#    y = model_trtmo(data)
#torch.cuda.current_stream().synchronize()
#t1 = time.time()

#print(50.0 / (t1 - t0))

Next, let's define a function that will preprocess the image, which is originally in BGR8 / HWC format.

In [9]:
import cv2
import torchvision.transforms as transforms
import PIL.Image

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda()
device = torch.device('cuda')

def preprocess(image):
    global device
    device = torch.device('cuda')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device)
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Next, we'll define two callable classes that will be used to parse the objects from the neural network, as well as draw the parsed objects on an image.

In [10]:
from trt_pose.draw_objects import DrawObjects
from trt_pose.parse_objects import ParseObjects

parse_objects = ParseObjects(topology)
draw_objects = DrawObjects(topology)

Assuming you're using NVIDIA Jetson, you can use the [jetcam](https://github.com/NVIDIA-AI-IOT/jetcam) package to create an easy to use camera that will produce images in BGR8/HWC format.

If you're not on Jetson, you may need to adapt the code below.

In [11]:
#from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

#camera = USBCamera(width=WIDTH, height=HEIGHT, capture_fps=30)
camera = CSICamera(width=WIDTH, height=HEIGHT, capture_fps=30)

camera.running = True

Next, we'll create a widget which will be used to display the camera feed with visualizations.

In [12]:
import ipywidgets
from IPython.display import display

image_w = ipywidgets.Image(format='jpeg')

display(image_w)

Image(value=b'', format='jpeg')

Finally, we'll define the main execution loop.  This will perform the following steps

1.  Preprocess the camera image
2.  Execute the neural network
3.  Parse the objects from the neural network output
4.  Draw the objects onto the camera image
5.  Convert the image to JPEG format and stream to the display widget

In [13]:
def execute(change):
    image = change['new']
    data = preprocess(image)
    cmap, paf = model_trt(data)
    cmap, paf = cmap.detach().cpu(), paf.detach().cpu()
    counts, objects, peaks = parse_objects(cmap, paf)#, cmap_threshold=0.15, link_threshold=0.15)
    draw_objects(image, counts, objects, peaks)
    image_w.value = bgr8_to_jpeg(image[:, ::-1, :])
    return peaks

If we call the cell below it will execute the function once on the current camera frame.

In [14]:
peak=execute({'new': camera.value})

In [78]:
#Classifier
import numpy as np
import time
counter=0
k=0
TH=0.17
Data=np.zeros((10000,37))
status=[]
display(image_w)
while (1==1):
    peak=execute({'new': camera.value})
    for i in range (18):
        Data[counter][k:k+2]=peak[0][i][0]
        k=k+2

    if (Data[counter][8]==0 and Data[counter][12]==0) or (Data[counter][6]==0 and Data[counter][10]==0):
        print("Not detected")
        status.append("Not detected")
    elif (Data[counter][24]==0 and Data[counter][22]==0):
        print("Seated1")
        status.append("Seated1")
    elif (Data[counter][24]!=0 and Data[counter][22]==0):
        if Data[counter][28]!=0:
            if (Data[counter][28]-Data[counter][24])>=TH:
                print("Standing position1")
                status.append("Standing position1")
            else:
                print("seated2")
                status.append("seated2")
        else:
                print("seated3")
                status.append("seated3")
    elif (Data[counter][24]==0 and Data[counter][22]!=0):
        if Data[counter][26]!=0:
            if (Data[counter][26]-Data[counter][22])>=TH:
                print("Standing position2")
                status.append("Standing position2")
            else:
                print("seated4")
                status.append("seated4")
        else:
            print("seated5") 
            status.append("seated5")
    elif (Data[counter][24]!=0 and Data[counter][22]!=0):
        if Data[counter][28]!=0:
            if Data[counter][28]-Data[counter][24]>=TH:
                    print("Standing position3")
                    status.append("Standing position3")
            else:
                    print("seated6")
                    status.append("seated6")
        else:
            print("seated7")
            status.append("seated7")
    counter=counter+1
    k=0
    time.sleep(3)                
    
    #print(Data)
    
    

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Not detected
Not detected
Not detected
Standing position3
Standing position3
seated6
seated6
seated6
Standing position3
Standing position3
Standing position3
Standing position3
Standing position3
Not detected


KeyboardInterrupt: 

Call the cell below to attach the execution function to the camera's internal value.  This will cause the execute function to be called whenever a new camera frame is received.

In [ ]:
#camera.observe(execute, names='value')

Call the cell below to unattach the camera frame callbacks.

In [ ]:
#camera.unobserve_all()

In [44]:
len(peak[0][0][0])

100

In [63]:
print (Data[6])

[0.07400861 0.80230635 0.0184967  0.80941767 0.03887404 0.77398503
 0.         0.         0.09922341 0.71458858 0.27781704 0.90192187
 0.29729861 0.66907197 0.         0.         0.65427065 0.68532455
 0.         0.         0.68851805 0.79433441 0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.2875351  0.79057455]


In [42]:
print(a)

tensor([[[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        ...,

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]]])


In [80]:
Data[3]

array([0.13395573, 0.55054271, 0.11298265, 0.56084597, 0.11574358,
       0.53489506, 0.12122156, 0.5758338 , 0.12902947, 0.51031458,
       0.26490527, 0.61343127, 0.27306336, 0.48442221, 0.43060941,
       0.64638191, 0.43770513, 0.45426723, 0.55594426, 0.6317423 ,
       0.54705089, 0.46349493, 0.59935653, 0.59433234, 0.60111821,
       0.50688636, 0.8666274 , 0.60667866, 0.88939047, 0.5156219 ,
       0.        , 0.        , 0.        , 0.        , 0.26916033,
       0.54776794, 0.        ])

In [79]:
status

['Not detected',
 'Not detected',
 'Not detected',
 'Standing position3',
 'Standing position3',
 'seated6',
 'seated6',
 'seated6',
 'Standing position3',
 'Standing position3',
 'Standing position3',
 'Standing position3',
 'Standing position3',
 'Not detected']

In [81]:
Data[3][12]-Data[3][24]

-0.32805484533309937

In [82]:
Data[3][28]-Data[3][24]

0.28827226161956787

In [83]:
Data[3][28]-Data[3][32]

0.8893904685974121